# Metamorph Controler for Python 

### Definition of a setting dictionary to store the values of the parameters. 
Here the dictionary has some values already preset. This dictionary is defined as a global variable, being called in and out of functions

In [1]:
global setting
setting={"path":'',"name":'',"laser_405":'0',"laser_488":'0',"laser_561":'0',"laser_642": '0'}
setting["obs"]='CSUTRANS'
setting["act"]=''
setting["exp"]='200'
setting["g"]='400'

### Importation of the libraries and functions necessary to run the controler

In [2]:
# libraries 
from tkinter import *
from tkinter import ttk
from tkinter.messagebox import showinfo
from tkinter.ttk import Combobox
import matplotlib.pyplot as plt
import numpy as np
import math 
import time
import tifffile as tif
from PIL import Image
import pandas as pd
import json
from PIL.TiffTags import TAGS
import PIL
from scipy.optimize import curve_fit

# Connection to MM
import clr
clr.AddReference('D:\\Users\\Iris\\Interop.MMAppLib.dll')
import MMAppLib
core=MMAppLib.UserCallClass()
#print(dir(core))
jnl_path='C:\\MM\\app\\mmproc\\journals\\s.JNL' # path to the journal taking a snapshot and saving it somewhere
path_tmp='C:\\TEMP\\' #location of the temporary image file generated during acquisition - can be modified in MM

 
#positions 
name_csv=path_tmp+'current_position'+'.csv'
pos_experiment=pd.DataFrame(np.array([[0,0,0]]),columns=['x', 'y', 'z'])
pos_experiment.drop([0], inplace = True)
pos_experiment.to_csv(name_csv, index = False)

# functions 
#coord=[x,y] in um
def set_pos(coord):
    core.SetMMVariable('Device.Stage.XPosition',coord[0])
    core.SetMMVariable('Device.Stage.YPosition',coord[1])
    core.SetMMVariable('Device.Focus.CurPos',coord[2])
    return 

#add the current position to a dataframe
def add_pos(all_pos):  
    a,b,x=core.GetMMVariable('Device.Stage.XPosition',0)
    a,b,y=core.GetMMVariable('Device.Stage.YPosition',0)
    a,b,z=core.GetMMVariable('Device.Stage.CurPos',0)
    current_pos=pd.DataFrame(np.array([[x,y,z]]),columns=['x', 'y', 'z']) 
    all_pos_updated=all_pos.append(current_pos)    
    return all_pos_updated


#set the wavelenght (using a predefined channel)
def set_wl(chan):
    core.SetMMVariable('Device.Illumination.Setting',chan) 
    return 
        
#set exposure (number in ms)
def set_exposure(exp):
    core.SetMMVariable('Device.Illumination.Exposure',exp)
    return 

def get_exposure():
    a,b,exp=core.GetMMVariable('Device.Illumination.Exposure',0)
    return exp

#set the gain  
def set_gain(g): 
    core.SetMMVariable('Device.Illumination.EMGain',g)
    return 

def get_gain():
    g=core.GetMMVariable('Device.Illumination.EMGain',0)
    return g
def set_laser(laser_405,laser_488,laser_561,laser_642):
    core.SetMMVariable('Component.Laser_DAQ_Intensity_405.Position',laser_405)
    core.SetMMVariable('Component.Laser_DAQ_Intensity_488.Position',laser_488)
    core.SetMMVariable('Component.Laser_DAQ_Intensity_561.Position',laser_561)
    core.SetMMVariable('Component.Laser_DAQ_Intensity_642.Position',laser_642)
    
#image acquisition
def acquire(jnl_path,path_tmp):
    #run journal doing the acquisition
    core.RunJournal(jnl_path)
    #open the open image as tif
    pixvals=np.array((Image.open(path_tmp+'temp.tif')))
    return pixvals

#image acquisition + save as .tif
def acquire_save(name,jnl_path,path_tmp):
    #run journal doing the acquisition
    core.RunJournal(jnl_path)
    #open the temporary image saved as a .tif
    pixvals=np.array((Image.open(path_tmp+'temp.tif')))
    #save it as a definitive .tif file in the user's file location 
    tif.imwrite(name,pixvals,append=True)
    
#because there is an issue with the coordinate formating when doing loops 
def mat_conversion(mat): 
    x=float(mat[0])
    y=float(mat[1])
    z=float(mat[2])
    return [x,y,z]

#convert the values of a dataframe to a list of positions
def dataframe_to_coordinates(df):
    coord_m=[]
    for n in range(df.shape[0]):
        coord_n=[]
        for element in df.iloc[n]:
            coord_n.append(element)
        coord_m.append(coord_n)
    return coord_m

# normalize a matrix by its highest value 
def normalization(matrix):
    N,M=np.shape(matrix)
    new_mat=np.zeros([N,M])
    mat_max=np.amax(matrix)
    for i in range(N):
        for j in range(M):
            new_mat[i][j]=matrix[i][j]/mat_max            
    return new_mat


def coord_from_str(coord_str):
    c=[]
    for n in range(1,len(coord_str)):
        if coord_str[n]==',':
            c.append(n)
    x=int(coord_str[1:c[0]])
    y=int(coord_str[(c[0]+1):c[1]])
    z=int(coord_str[(c[1]+1):(len(coord_str)-1)])
    return [x,y,z]

def coord_to_str(coord):
    string=''
    for n in range(len(coord)):
        string+=str(coord[n])
        if n!=len(coord)-1:
            string+=','
    return string

def timestep_to_list(string):
    li = list(string.split(","))
    L=[]
    for element in li:
        L.append(int(element))
    return L

def acquisition(setting):
    print(setting)
    '''Get parameters '''
    name=setting['path']+'\\'+setting['name']+'_' #File path and name
    laser_405=int(setting['laser_405']) #laser intensities
    laser_488=int(setting['laser_488']) 
    laser_561=int(setting['laser_561'])
    laser_642=int(setting['laser_642'])
    wl_obs=setting['obs'] #obs and act wavelenght 
    wl_act=setting['act']
    exp=int(setting['exp']) #exposure
    G=int(setting['g']) #gain
    s_type=setting['type']
    if s_type=='list':
        t_observation=timestep_to_list(setting["step list"])
        t_activation=timestep_to_list(setting["pulse list"])
    elif s_type=='input':
        step=int(setting['step']) #time lapse parameters 
        t_final=int(setting['tf'])
        act_pulse=int(setting['pulse'])# duration of 1 activation pulse
        act_lenght=int(setting['act_L'])#duration of total time of activation
        act_start=int(setting['act_t0']) #start time of activation
        t_activation=[act_start+t for t in range(0,act_lenght,act_pulse)]
        t_observation=[t for t in range(0,t_final+1,step)]
    coord=setting["coord"]
    ''' save Metadata '''
    coord_str='x='+str(coord[0])+', y='+str(coord[1])+', z='+str(coord[2])
    if s_type=='input':
        if wl_act:
            MD=np.array(['Position: '+coord_str,'Acquisition Channel: '+wl_obs,'Activation Channel: '+wl_act,'Step of timelaps: '+str(step),'total acquisition time : '+str(t_final),'t0 activation: '+str(act_start),'tf activation: '+str(act_start+act_lenght),'Exposure: '+str(exp),'Gain: '+str(G),'Laser Intensity 405: '+str(laser_405),'Laser Intensity 488: '+str(laser_488),'Laser Intensity 561: '+str(laser_561),'Laser Intensity 642: '+str(laser_642)])
        else:
            MD=np.array(['Position: '+coord_str,'Acquisition Channel: '+wl_obs,'Exposure: '+str(exp),'Gain: '+str(G),'Laser Intensity 405: '+str(laser_405),'Laser Intensity 488: '+str(laser_488),'Laser Intensity 561: '+str(laser_561),'Laser Intensity 642: '+str(laser_642)])
    elif s_type=='list':
        if wl_act:
            MD=np.array(['Position: '+coord_str,'Acquisition Channel: '+wl_obs,'Activation Channel: '+wl_act,'Step of timelaps: '+setting["step list"],'Activation pulse: '+setting["pulse list"],'Exposure: '+str(exp),'Gain: '+str(G),'Laser Intensity 405: '+str(laser_405),'Laser Intensity 488: '+str(laser_488),'Laser Intensity 561: '+str(laser_561),'Laser Intensity 642: '+str(laser_642)])
        else:
            MD=np.array(['Position: '+coord_str,'Acquisition Channel: '+wl_obs,'Exposure: '+str(exp),'Gain: '+str(G),'Laser Intensity 405: '+str(laser_405),'Laser Intensity 488: '+str(laser_488),'Laser Intensity 561: '+str(laser_561),'Laser Intensity 642: '+str(laser_642)])
    MD_name=name+'_obs'+str(wl_obs)+'_metadata.txt'
    with open(MD_name, 'w') as f:
        for line in MD:
            f.write(line)
            f.write('\n')
    ''' prepare the microscope'''
    set_laser(laser_405,laser_488,laser_561,laser_642)
    set_exposure(exp)
    set_gain(G)
    set_pos(coord)
    ''' start the acquisition'''
    t0=round(time.time())
    time_points_obs=np.array([round(t0+t) for t in t_observation])
    time_points_act=[round(t0+t) for t in t_activation]
    local_time=round(time.time())
    t=1
    counter=0
    limit=len(time_points_obs)+len(time_points_act)
    while counter<limit:
        local_time=round(time.time())
        if local_time in time_points_act:
            set_wl(wl_act)
            acquire(jnl_path,path_tmp)
            print('=> => => act',local_time-t0)
            counter+=1
                
        
        if local_time in time_points_obs:
            set_wl(wl_obs)
            print('obs',local_time-t0)
            counter+=1
            #save timelapse as stack 
            stack_name_n=name+'_'+'obs'+str(wl_obs)+'.tif'
            acquire_save(stack_name_n,jnl_path,path_tmp)
        time.sleep(t)
    print("done")
    return 


print('good to go')

good to go


### Load up the GUI and start an acquisition

In [4]:
class MyWindow:
    def __init__(self, win):
        # labels 
        self.lbl0=Label(win, text='File',font=("Arial", 15))
        self.lbl1=Label(win, text='Path',font=("Arial", 10))
        self.lbl2=Label(win, text='Name',font=("Arial", 10))
        self.lbl4=Label(win, text='405 nm (%)',font=("Arial", 10))
        self.lbl5=Label(win, text='488 nm (%)',font=("Arial", 10))
        self.lbl6=Label(win, text='561 nm (%)',font=("Arial", 10))
        self.lbl7=Label(win, text='642 nm (%)',font=("Arial", 10))
        self.lbl8=Label(win, text='Laser Settings',font=("Arial", 15))
        self.lbl9=Label(win, text='Observation channel',font=("Arial", 10))     
        self.lbl10=Label(win, text='Activation channel',font=("Arial", 10))
        self.lbl13=Label(win, text='Image settings',font=("Arial", 15))    
        self.lbl11=Label(win, text='Exposure (ms) ',font=("Arial", 10))     
        self.lbl12=Label(win, text='Gain ',font=("Arial", 10))
        self.lbl14=Label(win, text='Time lapse settings',font=("Arial", 15))     
        self.lbl15=Label(win, text='Time steps (s) ',font=("Arial", 10))
        self.lbl16=Label(win, text='Total time (s)',font=("Arial", 10))
        self.lbl17=Label(win, text='Pulse (s)',font=("Arial", 10))
        self.lbl18=Label(win, text='Start (s)',font=("Arial", 10))
        self.lbl19=Label(win, text='Duration (s)',font=("Arial", 10))
        self.lbl20=Label(win, text='Activation',font=("Arial", 10))
        self.lbl21=Label(win, text='Acquire current position',font=("Arial", 15))
        self.lbl22=Label(win, text='will acquire the current position from MM when clicking set setting',font=("Arial", 10))
        # written entries 
        self.t1=Entry(bd=3)
        self.t2=Entry()
        self.t4=Entry()
        self.t5=Entry()
        self.t6=Entry()
        self.t7=Entry()
        self.t11=Entry()
        self.t12=Entry()
        self.t15=Entry()
        self.t16=Entry()
        self.t17=Entry()
        self.t18=Entry()
        self.t19=Entry()
         # rolling window entries 
        var1 = StringVar()
        var1.set("")
        data=("","CSUTRANS", "CSU405", "CSU488", "CSU561","CSU640")
        self.cb1=Combobox(window, values=data)
        self.cb2=Combobox(window, values=data)
        #restore previous settings 
        self.t1.insert(END, setting['path'])
        self.t2.insert(END, setting['name'])
        self.t4.insert(END, setting["laser_405"])
        self.t5.insert(END, setting["laser_488"])
        self.t6.insert(END, setting["laser_561"])
        self.t7.insert(END, setting["laser_642"])
        self.t11.insert(END, setting['exp'])
        self.t12.insert(END, setting['g'])
        if 'step' in setting.keys():
            self.t15.insert(END, setting['step'])
            self.t16.insert(END, setting['tf'])
            self.t17.insert(END, setting['pulse'])
            self.t18.insert(END, setting['act_t0'])
            self.t19.insert(END, setting['act_L'])
              
        obs=setting['obs']
        act=setting['act']
        n1= data.index(obs)
        n2= data.index(act)
        self.cb1.current(n1)
        self.cb2.current(n2)
        #setting up the button 
        self.b1=Button(win, text='Set settings', command=self.set_setting)
        self.b2 = Button(win,text ="Advanced settings",command = self.openNewWindow)
        # placements 
        ''' file'''
        self.lbl0.place(x=20, y=0)
        self.lbl1.place(x=20, y=25)
        self.t1.place(x=80, y=25,width=300)
        self.lbl2.place(x=20, y=60)
        self.t2.place(x=80, y=60,width=300)
        '''laser power '''
        self.lbl4.place(x=20, y=130)
        self.t4.place(x=100, y=130,width=100)
        self.lbl5.place(x=20, y=160)
        self.t5.place(x=100, y=160,width=100)
        self.lbl6.place(x=20, y=190)
        self.t6.place(x=100, y=190,width=100)
        self.lbl7.place(x=20, y=220)
        self.t7.place(x=100, y=220,width=100)
        self.lbl8.place(x=20, y=95)
        ''' channels '''
        self.lbl9.place(x=250, y=110)
        self.lbl10.place(x=250, y=180)
        '''Image setting '''
        self.lbl11.place(x=20, y=285)
        self.t11.place(x=120, y=285,width=100)
        self.lbl12.place(x=20, y=315)
        self.t12.place(x=120, y=315,width=100)
        self.lbl13.place(x=20, y=250)
        ''' time lapse'''
        self.lbl14.place(x=20, y=340)
        self.lbl15.place(x=20, y=370)
        self.t15.place(x=120, y=370,width=100)
        self.lbl16.place(x=20, y=400)
        self.t16.place(x=120, y=400,width=100)
        self.lbl17.place(x=250, y=370)
        self.t17.place(x=350, y=370,width=100)
        self.lbl18.place(x=250, y=400)
        self.t18.place(x=350, y=400,width=100)
        self.lbl19.place(x=250, y=430)
        self.t19.place(x=350, y=430,width=100)
        self.lbl20.place(x=250, y=350)
        '''position'''
        self.lbl21.place(x=20, y=460)
        self.lbl22.place(x=20, y=490)
        ''' buttons and rolling windows'''
        self.b1.place(x=350, y=520)
        self.b2.place(x=200, y=520)
        self.cb1.place(x=250, y=140)
        self.cb2.place(x=250, y=210)
    def openNewWindow(self):
        # Toplevel object which will
        # be treated as a new window
        global newWindow
        self.newWindow = Toplevel()

        # sets the title of the
        # Toplevel widget
        self.newWindow.title("Advanced setting")

        # sets the geometry of toplevel
        self.newWindow.geometry("500x400")

        # A Label widget to show in toplevel
        self.lb30=Label(self.newWindow, text='Advanced settings',font=("Arial", 15))
        self.lb31=Label(self.newWindow, text='Observation time points (as a list)',font=("Arial", 10))
        self.lb32=Label(self.newWindow, text='Activation time points (as a list)',font=("Arial", 10))
        self.t31=Entry(self.newWindow)
        self.t32=Entry(self.newWindow)
        self.lb30.place(x=20, y=0)
        self.lb31.place(x=20, y=40)
        self.t31.place(x=20, y=70,width=450)
        self.lb32.place(x=20, y=100)
        self.t32.place(x=20, y=130,width=450)
        self.b3= Button(self.newWindow, text="Set Advanced Settings", command=self.set_advanced_setting)
        self.b3.place(x=300, y=350)
    def set_advanced_setting(self):
        global num31
        global num32
        num31=self.t31.get()
        num32=self.t32.get()
        self.newWindow.destroy()
    def set_setting(self):
        num1=self.t1.get()
        num2=self.t2.get()
        num4=self.t4.get()
        num5=self.t5.get()
        num6=self.t6.get()
        num7=self.t7.get()
        num11=self.t11.get()
        num12=self.t12.get()
        num15=self.t15.get()
        num16=self.t16.get()
        num9=self.cb1.get()
        num10=self.cb2.get()
        num17=self.t17.get()
        num18=self.t18.get()
        num19=self.t19.get()
        #get current coordinates
        pos_experiment=pd.read_csv(name_csv)
        pos_experiment=add_pos(pos_experiment)
        coord_m=dataframe_to_coordinates(pos_experiment)
        r=len(coord_m)
        pos_experiment.to_csv(name_csv, index = False)
        file_name=num1+'\\'+num2
        global setting
        setting={}
        setting={"path":num1,"name":num2,"laser_405":num4,"laser_488":num5,"laser_561":num6,"laser_642": num7}
        setting["obs"]=num9
        setting["act"]=num10
        setting["exp"]=num11
        setting["g"]=num12
        if num31!='' and num15=='':
            setting["type"]='list'
            setting["step list"]=num31
            setting["pulse list"]=num32
        else:
            setting["type"]='input'
            setting["step"]=num15
            setting["tf"]=num16
            setting["pulse"]=num17
            setting["act_t0"]=num18
            setting["act_L"]=num19            
        setting["coord"]=coord_m[r-1]
        for key in setting.keys():
            c=0
            if setting[key]=='':
                c+=1
        if c!=0: 
            showinfo("Error", "Please set up the acquisition \n One or mutiple parameters may be missing")
        else: 
            text="file_name : "+file_name+" \n"
            text=text+"Laser settings : 405="+num4+'% - 488='+num5+'% - 561='+num6+'% - 640='+num7+"% \n"
            text=text+'Observation='+num9+'nm - activation='+num10+'nm \n'
            text=text+"Exposure : "+num11+ " - Gain : "+num12+" \n"
            if setting["type"]=='list':
                text=text+"Time lapse setting : steps="+num31+"\n"
                text=text+"    pulses "+num32+"\n"
            else:
                text=text+"Time lapse setting : steps="+num15+"s - total time="+num16+"s \n"
                text=text+"    pulse every "+num17+ " from t="+num18+" to t="+num19+"\n"
                
            text=text+"coordinates: "+coord_to_str(coord_m[r-1])
            showinfo("Setting", text)
            

        
# create the root window

window=Tk()
mywin=MyWindow(window)
window.title('MM controler')
window.geometry("500x650+10+10")
# export the settings to dictionary 
b2= Button(window, text="Start acquisition", command=window.destroy)
b2.place(x=350, y=550)
window.mainloop()

acquisition(setting)


1,10,15,17,19,21,30,35 7,8,9,22,23,24
{'path': 'D:\\Users\\Iris\\test python', 'name': 'test_130422_2', 'laser_405': '0', 'laser_488': '10', 'laser_561': '10', 'laser_642': '0', 'obs': 'CSU488', 'act': 'CSU561', 'exp': '200', 'g': '400', 'type': 'input', 'step': '6', 'tf': '30', 'pulse': '2', 'act_t0': '15', 'act_L': '21', 'coord': [-61, -808, 0]}
obs 0
obs 6
obs 12
=> => => act 15
=> => => act 17
obs 18
=> => => act 19
=> => => act 21
=> => => act 23
=> => => act 25
=> => => act 27
=> => => act 29
obs 30
=> => => act 33
=> => => act 35


KeyboardInterrupt: 